# Mongodb

Dans ce TP nous allons utiliser la librairie pymongo, pour créer une base de données dans mongodb avec python puis la questionner.

### Data
Voici des données au format csv, elles sont une partie d'un dataset recensant les crimes au Etats-Unis entre 1984 et 2014.

https://drive.google.com/file/d/10z7kUXDO4BHffJ6ZfVc42CgIs5558vGd/view?usp=sharing

### Création de la BDD

In [21]:
import pandas as pd
from pymongo import MongoClient
import json
import streamlit as st

Créez une fonction python pour passer ce csv dans une base de données mongo, chaque lignes devra être un document.

### Requêter la BDD

Créez un connecteur pour votre BDD

In [22]:
client = MongoClient('localhost', 27017)
db = client.crimes
table = db.documents

In [10]:
# Lecture du csv
df = pd.read_csv('Crime.csv')

In [46]:
# En une seule fois
if table.count_documents({}) == 0:    
    records = df.to_dict(orient='records')
    table.insert_many(records)

In [47]:
# Ligne par ligne
if table.count_documents({}) == 0:
    json_docs = df.apply(lambda x: x.to_json(), axis=1)
    for document in json_docs:

        json_doc = json.loads(document)
        table.insert_one(json_doc)

##### Quelles sont les armes utilisées par les criminel?
Retournez par rapport à tous les documents les valeurs uniques de la clef weapon.

In [13]:
table.distinct("Weapon")

list(table.aggregate([
    { "$group": {
        "_id": "$Weapon", "count" : {"$sum": 1}
    }},
    { "$sort": {"count": -1}}
]))

[{'_id': 'Handgun', 'count': 45748},
 {'_id': 'Knife', 'count': 19586},
 {'_id': 'Blunt Object', 'count': 11150},
 {'_id': 'Shotgun', 'count': 7419},
 {'_id': 'Rifle', 'count': 4968},
 {'_id': 'Firearm', 'count': 3629},
 {'_id': 'Unknown', 'count': 3134},
 {'_id': 'Strangulation', 'count': 1742},
 {'_id': 'Fire', 'count': 1254},
 {'_id': 'Suffocation', 'count': 600},
 {'_id': 'Drowning', 'count': 265},
 {'_id': 'Gun', 'count': 212},
 {'_id': 'Drugs', 'count': 101},
 {'_id': 'Poison', 'count': 80},
 {'_id': 'Explosives', 'count': 61},
 {'_id': 'Fall', 'count': 50}]

##### Combien de crimes ont été commis en 1980?

In [54]:
table.count_documents({"Year":1980})

23092

##### Combien de crimes ont été commis par des hommes au Texas?

In [84]:
table.count_documents({"Perpetrator Sex": "Male", "State": "Texas"})

6160

##### Combien de crimes ont été commis par chaque sexe en Alaska?

In [114]:
maleAndFemalePerpetratorsInAlaska = table.aggregate([
    { "$match": { "State": "Alaska" }},
    { "$group": {"_id": "$Perpetrator Sex", "crimes commis": {"$sum": 1}}},
])

list(maleAndFemalePerpetratorsInAlaska)

[{'_id': 'Male', 'crimes commis': 226},
 {'_id': 'Unknown', 'crimes commis': 63},
 {'_id': 'Female', 'crimes commis': 28}]

##### Combien y a-t'il eu de victimes dans chaque état?

In [19]:
victimInEachState = table.aggregate([
    { "$group" : {
        "_id": "$State", "victimes": {"$sum": "$Victim Count"}
    }},
    { "$sort": { "victimes": -1}}
])

list(victimInEachState)

[{'_id': 'California', 'victimes': 1678},
 {'_id': 'New York', 'victimes': 1182},
 {'_id': 'Florida', 'victimes': 888},
 {'_id': 'Texas', 'victimes': 810},
 {'_id': 'New Jersey', 'victimes': 558},
 {'_id': 'Michigan', 'victimes': 546},
 {'_id': 'Illinois', 'victimes': 512},
 {'_id': 'Pennsylvania', 'victimes': 420},
 {'_id': 'Ohio', 'victimes': 354},
 {'_id': 'Maryland', 'victimes': 256},
 {'_id': 'Massachusetts', 'victimes': 256},
 {'_id': 'Missouri', 'victimes': 250},
 {'_id': 'Louisiana', 'victimes': 230},
 {'_id': 'Washington', 'victimes': 216},
 {'_id': 'Connecticut', 'victimes': 198},
 {'_id': 'Alaska', 'victimes': 188},
 {'_id': 'Indiana', 'victimes': 186},
 {'_id': 'Tennessee', 'victimes': 154},
 {'_id': 'Virginia', 'victimes': 148},
 {'_id': 'Nevada', 'victimes': 132},
 {'_id': 'Arizona', 'victimes': 126},
 {'_id': 'Kentucky', 'victimes': 122},
 {'_id': 'Kansas', 'victimes': 120},
 {'_id': 'North Carolina', 'victimes': 116},
 {'_id': 'Arkansas', 'victimes': 114},
 {'_id': 'Ala

### Bonus

Installez la librairie streamlit
Créez un dashboard qui donne la posibilité de séléctionner une année pour retourner un barplot du nombre de crime commis avec chaque arme.

In [26]:
victimWithEachWeapon = table.aggregate([
    {"$match": {"Year": 1980}},
    { "$group" : {
        "_id": "$Weapon", "victimes": {"$sum": "$Victim Count"}
    }}
])
    
list(victimWithEachWeapon)

[{'_id': 'Gun', 'victimes': 12},
 {'_id': 'Strangulation', 'victimes': 38},
 {'_id': 'Handgun', 'victimes': 896},
 {'_id': 'Poison', 'victimes': 2},
 {'_id': 'Unknown', 'victimes': 122},
 {'_id': 'Fall', 'victimes': 2},
 {'_id': 'Blunt Object', 'victimes': 80},
 {'_id': 'Drowning', 'victimes': 4},
 {'_id': 'Rifle', 'victimes': 204},
 {'_id': 'Explosives', 'victimes': 32},
 {'_id': 'Fire', 'victimes': 524},
 {'_id': 'Knife', 'victimes': 204},
 {'_id': 'Firearm', 'victimes': 116},
 {'_id': 'Shotgun', 'victimes': 154},
 {'_id': 'Suffocation', 'victimes': 20},
 {'_id': 'Drugs', 'victimes': 0}]